In [1]:
import torch
print(torch.cuda.is_available())
print(torch.rand(2, 2))

True
tensor([[0.2268, 0.8954],
        [0.5750, 0.1618]])


In [2]:
cpu_tensor = torch.rand(2, 2)
cpu_tensor.device

device(type='cpu')

In [3]:
# Прокиним тензор в GPU
gpu_tensor = cpu_tensor.to('cuda')
gpu_tensor.device

device(type='cuda', index=0)

[Документация pytorch](https://pytorch.org/docs/stable/index.html)

[Тензоры в pytorch](https://pytorch.org/docs/stable/index.html)

In [4]:
tens = torch.rand(3, 3)

In [5]:
# максимальный элемент в тензоре
tens.max()

tensor(0.9724)

In [6]:
# значение
tens.max().item()

0.9723513722419739

In [7]:
tens = torch.tensor([[0, 1, 1], [2, 0, 0], [1, 3, 2]])

In [8]:
# изменим тип тензора
tens.type()

'torch.LongTensor'

In [9]:
tens_f = tens.to(dtype=torch.float32)
tens_f.type()

'torch.FloatTensor'

In [10]:
# для inplace методов определены именя, завершающиеся на _
tens = torch.rand(3, 3)
tens.log2()

tensor([[-1.5025, -1.5619, -0.0250],
        [-0.3265, -1.3120, -0.0554],
        [-0.0248, -3.1707, -0.4862]])

In [11]:
tens

tensor([[0.3529, 0.3387, 0.9828],
        [0.7975, 0.4028, 0.9623],
        [0.9829, 0.1110, 0.7139]])

In [12]:
# а теперь на месте
tens.log2_()

tensor([[-1.5025, -1.5619, -0.0250],
        [-0.3265, -1.3120, -0.0554],
        [-0.0248, -3.1707, -0.4862]])

In [13]:
tens

tensor([[-1.5025, -1.5619, -0.0250],
        [-0.3265, -1.3120, -0.0554],
        [-0.0248, -3.1707, -0.4862]])

In [14]:
# измененеие формы представления тензора
flat_tensor = torch.rand(784)
flat_tensor.shape

torch.Size([784])

In [15]:
# отображает flat_tensor в новом виде. Если в flat_tensor внести изменения - изменится и продукт view()
viewed_tensor = flat_tensor.view(1, 28, 28)
viewed_tensor.shape

torch.Size([1, 28, 28])

In [16]:
# тоже самое, но с проверкой смежности после внесенных изменений
reshaped_tensor = flat_tensor.reshape(1, 28, 28)
reshaped_tensor.shape

torch.Size([1, 28, 28])

In [17]:
# изменение рамзера тензора
hwc_tensor = torch.rand(640, 480, 1)
chw_tensor = hwc_tensor.permute(2, 0, 1)
chw_tensor.shape

torch.Size([1, 640, 480])

## Пример классификации изображений

### Предварительная обработка

Класс ImageFolder из torchvision позволяет создать набор данных, при условии, что данные хранятся в структуре, в которой каждый каталог представляет из себя маркировку.

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES=True

In [19]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

В данном случае:

- изображения ресайзятся в 64х64 (на самом деле для ускорения вычислений). Как правило считается на 224х224 или 229х229 ну или в зависимости от вычислительных возможностей
- перегоняются в тензор
- нормализуются средним стандартным отклонением ImageNet (для работы с реальнысми данными конечно стоит попробоывать посчитать эти значения непосредственно для дата-сета)

In [20]:
train_data_path = 'pytorch_learning/train/'
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.234, 0.225])
])
train_data = torchvision.datasets.ImageFolder(root=train_data_path, 
                                              transform=transform, 
                                              is_valid_file=check_image)

### Валидация и контрольные наборы

In [21]:
val_data_path = 'pytorch_learning/val/'
val_data = torchvision.datasets.ImageFolder(root=val_data_path, 
                                            transform=transform, 
                                            is_valid_file=check_image)

In [22]:
test_data_path = 'pytorch_learning/test/'
test_data = torchvision.datasets.ImageFolder(root=test_data_path, 
                                             transform=transform, 
                                             is_valid_file=check_image)

### Загрузчик данных

In [23]:
batch_size = 64
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)

### Простая полносвязная сеть 

три входа, три нейрона с крытом слое, два выхода, релу, софтмакс

In [24]:
class SimpleNet(nn.Module):
    
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 84) # Linear - полносвязный слой в Pytorch
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50, 2) # два класса на выходе
        
    def forward(self, x):
        x = x.view(-1, 12288) # трехмерное изображение в одномерный тензор
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
#         x = F.softmax(self.fc3(x))
        x = self.fc3(x) # софтмакс не нужен, т.к он включен в CrossEntropuLoss
        return x

In [25]:
simplenet = SimpleNet()

### Оптимизатор

Adam изменяет скорость обучения по прараметрам в зависимости от этих параметров. Оптимизация нужна для скорейшего нахождения миниума градиентов.

In [26]:
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

### GPU или CPU

In [27]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('cuda')
else:
    device = torch.device("cpu")
    print('cpu')

simplenet.to(device)

cuda


SimpleNet(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

### Обучение

In [28]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [29]:
train(simplenet, 
      optimizer,torch.nn.CrossEntropyLoss(), 
      train_data_loader,
      val_data_loader, 
      epochs=10, 
      device=device)

Epoch: 0, Training Loss: 3.24, Validation Loss: 9.73, accuracy = 0.36
Epoch: 1, Training Loss: 4.39, Validation Loss: 1.89, accuracy = 0.71
Epoch: 2, Training Loss: 1.36, Validation Loss: 1.94, accuracy = 0.55
Epoch: 3, Training Loss: 1.18, Validation Loss: 0.80, accuracy = 0.69
Epoch: 4, Training Loss: 0.53, Validation Loss: 0.98, accuracy = 0.65
Epoch: 5, Training Loss: 0.48, Validation Loss: 0.72, accuracy = 0.69
Epoch: 6, Training Loss: 0.33, Validation Loss: 0.91, accuracy = 0.65
Epoch: 7, Training Loss: 0.30, Validation Loss: 0.73, accuracy = 0.69
Epoch: 8, Training Loss: 0.24, Validation Loss: 0.78, accuracy = 0.69
Epoch: 9, Training Loss: 0.21, Validation Loss: 0.78, accuracy = 0.68


### Прогноз

In [30]:
labels = ['cat','fish']
def predictor(image_for_predict):
    img = Image.open(image_for_predict) 
    img = transform(img).to(device) # конвертим картинку в тензор и отправляем в GPU

    prediction = F.softmax(simplenet(img), dim=1) # получаем вероятность
    prediction = prediction.argmax() # индекс наибольшего тензора
    print(labels[prediction]) 

In [31]:
# cat
predictor("pytorch_learning/test/cat/2041806579_e4a7f31b32.jpg")

cat


In [32]:
# fish
predictor("pytorch_learning/test/fish/1609947018_bfa1fcd6b2.jpg")

cat


In [33]:
# fish
predictor("pytorch_learning/test/fish/2869804396_c4127ccec6.jpg")

fish


### Сохранение модели

torch.save() использует pickle

In [34]:
torch.save(simplenet, "pytorch_learning/tmp/simplenet") 
simplenet = torch.load("pytorch_learning/tmp/simplenet") 

/home/konstantin/anaconda3/envs/pytorch-gpu/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type SimpleNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Сохраняются параметрыи структура модели. Это неудобно, так как если позже модель меняется - сложно сделать частичный откат. Более распространенная схема - сохранить модель в стандартный словарь python, который будет содержать карты параметров каждого слоя модели. Чтобы восстановить модель из такого словаря необходимо создать экземпляр модели, а затем загрузитьо в модель параметры слоев из словаря.

метод state_dict() - [подробнее](https://pytorch.org/tutorials/recipes/recipes/what_is_state_dict.html)

Если задать параметр strict=False для load_state_dict, чтобы не получать ошибки, если в каких-то слоях не хватает параметров. Кроме того, так как это обычный словарь - его можно править вручную.

In [35]:
torch.save(simplenet.state_dict(), "pytorch_learning/tmp/simplenet")    

In [36]:
simplenet = SimpleNet()
simplenet_state_dict = torch.load("pytorch_learning/tmp/simplenet")
simplenet.load_state_dict(simplenet_state_dict)   

<All keys matched successfully>